In [17]:
from mordecai import Geoparser
import pandas as pd
import re


In [4]:
geo = Geoparser()

geo.geoparse("I took the tube from Oxford Circus to London Bridge, via Bank")



[{'country_conf': 0.611725,
  'country_predicted': 'GBR',
  'geo': {'admin1': 'England',
   'country_code3': 'GBR',
   'feature_class': 'S',
   'feature_code': 'BDG',
   'geonameid': '6619889',
   'lat': '51.50821',
   'lon': '-0.08763',
   'place_name': 'London Bridge'},
  'spans': [{'end': 51, 'start': 38}],
  'word': 'London Bridge'}]

In [10]:
df = pd.read_csv('doctables/docs.csv')
df.head()

,Unnamed: 0,PY,abstract,firstauthor,firstauthor_instituttion,id,title
0,0,2017,Adaptation is necessary to cope with or take a...,"Blanco, V","Univ Edinburgh, Inst Geog & Lived Environm, Sc...",761059,The importance of socio-ecological system dyna...
1,1,2013,Both climate extremes and agricultural disaste...,"Tao, FL","Chinese Acad Sci, Inst Geog Sci & Nat Resource...",365430,Changes in rice disasters across China in rece...
2,2,2013,Hydrological models have been widely used to a...,"Vansteenkiste, T","Katholieke Univ Leuven, Hydraul Div, BE-3001 L...",453489,Climate change impact on river flows and catch...
3,3,2009,Arguments about the appropriate discount rate ...,"Atkinson, G","Univ London London Sch Econ & Polit Sci, Dept ...",86654,"Siblings, Not Triplets: Social Preferences for..."
4,4,2018,The Regional Approaches to Climate Change for ...,"Roesch-McNally, GE","USDA Northwest Climate Hub, 3200 SW Jefferson ...",1801649,U.S. Inland Pacific Northwest Wheat Farmers' P...


In [24]:
place_mentions = []
for i,d in df.iterrows():
    t = d['abstract'] 
    t = t.split("Copyright (C)")[0] 
    t = re.split("\([C-c]\) [1-2][0-9]{3} Elsevier",t)[0] 
    t = t.split("Published by Elsevier")[0] 
    t = t.split("Copyright. (C)")[0] 
    t = re.split("\. \(C\) [1-2][0-9]{3} ",t)[0] 
    t = re.split("\. \(C\) Copyright",t)[0]     
    t = t + ". " + d['title']
    gp = geo.geoparse(t)
    for p in gp:
        if "geo" in p:
            p['doc_id'] = d['id']
            for key,value in p['geo'].items():
                p[key] = value
            del p["geo"]
            place_mentions.append(p)
    
    
place_df = pd.DataFrame.from_dict(place_mentions)
place_df.head()

,admin1,country_code3,country_conf,country_predicted,doc_id,feature_class,feature_code,geonameid,lat,lon,place_name,spans,word
0,NA,SWE,0.999931,SWE,761059,A,PCLI,2661886,62,15,Kingdom of Sweden,"[{'start': 548, 'end': 554}]",Sweden
1,NA,CHN,0.999981,CHN,365430,A,PCLI,1814991,35,105,People’s Republic of China,"[{'start': 338, 'end': 343}]",China
2,NA,CHN,0.999981,CHN,365430,A,PCLI,1814991,35,105,People’s Republic of China,"[{'start': 1197, 'end': 1202}]",China
3,NA,CHN,0.999981,CHN,365430,A,PCLI,1814991,35,105,People’s Republic of China,"[{'start': 1539, 'end': 1544}]",China
4,NA,BEL,0.999931,BEL,453489,A,PCLI,2802361,50.75,4.5,Kingdom of Belgium,"[{'start': 569, 'end': 576}]",Belgium


In [27]:
place_df.to_csv('/home/max/ownCloud/adaptation-map/place_mentions.csv',index=False)

In [28]:
place_df.shape

(89411, 13)